In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys, pickle, numpy as np

sys.path.append('/content/drive/MyDrive')

from mock_ledger import MockBlockchain  # your blockchain class

# Load the ledger that FL notebook saved
with open('/content/drive/MyDrive/ledger.pkl', 'rb') as f:
    ledger: MockBlockchain = pickle.load(f)

print("Loaded ledger from Drive")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded ledger from Drive


In [ ]:
# Choose the round ID you used when saving updates
round_id = 1

# 1. Get raw payloads (bytes) from the ledger
payloads = ledger.get_round_updates(round_id)
num_clients = len(payloads)
print("Number of client updates from ledger:", num_clients)

if num_clients == 0:
    raise RuntimeError(f"No updates found in ledger for round {round_id}")

# 2. Infer vector length D from first payload
first_vec = np.frombuffer(payloads[0], dtype=np.float32)
D = first_vec.shape[0]
print("Inferred update dimension D:", D)

# 3. Rebuild full (num_clients, D) array
updates = np.zeros((num_clients, D), dtype=np.float32)
updates[0] = first_vec
for i in range(1, num_clients):
    updates[i] = np.frombuffer(payloads[i], dtype=np.float32, count=D)

print("Reconstructed updates from ledger:", updates.shape)
# -> e.g. (3, 11137)


Number of client updates from ledger: 8
Inferred update dimension D: 22274
Reconstructed updates from ledger: (8, 22274)


In [ ]:
!pip install pyfhel

import math
import numpy as np
from Pyfhel import Pyfhel


In [ ]:
import pickle

# Load ledger file
with open("/content/drive/MyDrive/ledger.pkl", "rb") as f:
    ledger = pickle.load(f)

# Extract raw bytes for all client payloads for round 1
payloads = ledger.get_round_updates(1)

# Convert back to numpy arrays
updates = np.array([np.frombuffer(p, dtype=np.float64) for p in payloads])


In [ ]:
def init_he_ckks():
    HE = Pyfhel()
    HE.contextGen(
        scheme='CKKS',
        n=2**14,               # 16384
        scale=2**40,
        qi_sizes=[60, 40, 40, 60],  # required for your Pyfhel version
    )
    HE.keyGen()
    HE.relinKeyGen()
    HE.rotateKeyGen()
    return HE

HE = init_he_ckks()
print("CKKS context and keys generated.")

CKKS context and keys generated.


In [ ]:
def encrypt_vector_chunked(HE: Pyfhel, vec: np.ndarray, chunk_size: int):
    vec = np.array(vec, dtype=np.float64)
    chunks = []
    for start in range(0, len(vec), chunk_size):
        sub = vec[start:start+chunk_size]
        ptxt = HE.encodeFrac(sub)
        ctxt = HE.encryptPtxt(ptxt)
        chunks.append(ctxt)
    return chunks   # list of ciphertexts

def decrypt_chunks(HE: Pyfhel, ctxt_chunks, total_len: int):
    vals = []
    for c in ctxt_chunks:
        dec = HE.decryptFrac(c)
        vals.extend(dec)
    return np.array(vals[:total_len], dtype=np.float64)



In [ ]:
import math
chunk_size = 2**13   # 8192 slots for n=2**14
# Re-derive D based on the current 'updates' array, as it might have been redefined
D = updates.shape[1] # Use the actual D of the updates array that will be encrypted
num_chunks = math.ceil(D / chunk_size)

print("Chunk size:", chunk_size)
print("Num chunks per client:", num_chunks)

Chunk size: 8192
Num chunks per client: 2


In [ ]:
encrypted_updates = [
    encrypt_vector_chunked(HE, updates[i], chunk_size)
    for i in range(num_clients)
]

print(f"🔐 Encrypted {len(encrypted_updates)} client updates.")
print("Client 0 chunks:", len(encrypted_updates[0]))


🔐 Encrypted 8 client updates.
Client 0 chunks: 2


In [ ]:
for i, chunks in enumerate(encrypted_updates):
    if len(chunks) != num_chunks:
        raise ValueError(...)


In [ ]:
agg_chunks = []

for j in range(num_chunks):
    agg_ctxt = encrypted_updates[0][j]

    # add the same chunk index from other clients
    for i in range(1, num_clients):
        agg_ctxt += encrypted_updates[i][j]

    # average: multiply by scalar (1/num_clients)
    agg_ctxt *= (1.0 / num_clients)

    agg_chunks.append(agg_ctxt)

print(f"✅ Homomorphic aggregation complete over {num_chunks} chunks.")


✅ Homomorphic aggregation complete over 2 chunks.


In [ ]:
# Decrypt aggregated chunks into one long vector
delta_avg_he = decrypt_chunks(HE, agg_chunks, D)

# Plaintext reference
delta_avg_plain = updates.mean(axis=0)

# Compare
diff = np.abs(delta_avg_he - delta_avg_plain)
print("\n🔎 HE vs Plain Average")
print(" Max |HE - plain| :", diff.max())
print(" Mean |HE - plain|:", diff.mean())

print("\nFirst 5 HE values:   ", delta_avg_he[:5])
print("First 5 plain values:", delta_avg_plain[:5])



🔎 HE vs Plain Average
 Max |HE - plain| : 6.075758374711471e-09
 Mean |HE - plain|: 6.268848940911755e-10

First 5 HE values:    [ 0.00839089 -0.00194939  0.0005085   0.00312208  0.00257224]
First 5 plain values: [ 0.00839089 -0.00194939  0.0005085   0.00312208  0.00257224]


In [ ]:
# Here we store the plaintext average; you could also store an encrypted version
agg_bytes = delta_avg_plain.astype(np.float32).tobytes()
ledger.store_aggregate(round_id, agg_bytes)

# Save updated ledger back to Drive if you want
with open('/content/drive/MyDrive/ledger_with_agg.pkl', 'wb') as f:
    pickle.dump(ledger, f)

print("✅ Stored aggregate on blockchain mock and saved updated ledger_with_agg.pkl")


[Ledger] Stored aggregate: round=1, hash=75ecf239...
✅ Stored aggregate on blockchain mock and saved updated ledger_with_agg.pkl


In [ ]:
# Max and mean absolute error between HE and plaintext aggregation
error_max = float(diff.max())
error_mean = float(diff.mean())

print("Max absolute error :", error_max)
print("Mean absolute error:", error_mean)


Max absolute error : 6.075758374711471e-09
Mean absolute error: 6.268848940911755e-10
